In [14]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, utils

model = Sequential()

model.add(Conv2D(16, (48, 48), strides=(1, 1), padding='same', use_bias=False, input_shape=(128, 128, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (24, 24), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (12, 12), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (6, 6), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))
 
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
 
 
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
 
# train_datagen = ImageDataGenerator(rescale = 1./255)
 
# 데이터셋 불러오기
#train_datagen = ImageDataGenerator(rescale=1./255)
#                                   ,rotation_range=10,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.7,
#                                   zoom_range=[0.9, 2.2],
#                                   horizontal_flip=True,
#                                   vertical_flip=True,
#                                   fill_mode='nearest',
#                                   validation_split=0.33)
 
#training_set = train_datagen.flow_from_directory('./data3/train',
#                                                 shuffle=True,
#                                                 seed=13,
#                                                 target_size = (128, 128),
#                                                 batch_size = 3,
#                                                 class_mode = 'categorical',
#                                                 subset="training")
#validation_set = train_datagen.flow_from_directory('./data3/validation',
#                                                 shuffle=True,
#                                                 seed=13,
#                                                 target_size = (128, 128),
#                                                 batch_size = 3,
#                                                 class_mode = 'categorical',
#                                                 subset="validation")

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
            'data3/train',
            target_size = (128,128),
            batch_size = 3,
            class_mode = 'categorical')


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
            'data3/test',
            target_size = (128,128),
            batch_size = 3,
            class_mode = 'categorical')


validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
            'data3/validation',
            target_size = (128,128),
            batch_size = 3,
            class_mode = 'categorical')
csv_logger = CSVLogger('./log.csv', append=True, separator=';')

#hist = model.fit_generator(train_generator,
#                         steps_per_epoch = 50,
#                         epochs = 200,
#                         validation_data = validation_set,
#                         validation_steps = 10,
#                         callbacks=[early_stop])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 15,
    epochs = 100,
    callbacks=[csv_logger], 
    validation_data = validation_generator,
    validation_steps = 10)

model.summary()
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 128, 128, 16)      110592    
_________________________________________________________________
batch_normalization_53 (Batc (None, 128, 128, 16)      48        
_________________________________________________________________
activation_53 (Activation)   (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 64, 64, 32)        294912    
_________________________________________________________________
batch_normalization_54 (Batc (None, 64, 64, 32)      

KeyboardInterrupt: 

In [ ]:
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
           test_set,
            steps = 10)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 
print(output)
print(validation_set.filenames)
